#   Проект: "Сопоставление названий городов с унифицированными именами geonames"

Цель:

- сопоставляет произвольное geo название с унифицированными именами geonames для внутреннего использования Карьерным центром.

Задачи:
- Создать решение для подбора наиболее подходящих названий с geonames.Например Ереван -> Yerevan;
- На примере РФ и стран наиболее популярных для релокации - Беларусь, Армения, Казахстан, Кыргызстан, Турция, Сербия. Города с населением от 15000 человек (свозможностью масштабирования на сервере заказчика);
- Возвращаемые поля geonameid, name, region, country, cosine similarity;
- формат данных на выходе: список словарей, например [{dict_1}, {dict_2}, …. {dict_n}], где словарь - одна запись с указанными полями.

Задачи опционально:
- возможность настройки количества выдачи подходящих названий (например в параметрах метода);
- коррекция ошибок и опечаток. Например Моченгорск -> Monchegorsk;
- хранение в PostgreSQL данных geonames;
- хранение векторизованных промежуточных данных в PostgreSQL;
- предусмотреть методы для настройки подключения к БД;
- предусмотреть метод для инициализации класса (первичная векторизация geonames);
- предусмотреть методы для добавления векторов новых гео названий.

Результат:
1) тетрадка с решением задачи (описание проекта, исследование, методы решения).
2) python-скрипт, содержащий функцию (класс), для интеграции в систему Заказчика.

Описание данных:
Используемые таблицы с geonames:
- admin1CodesASCII
- alternateNamesV2
- cities15000
- countryInfo
- при необходимости любые другие открытые данные
- таблицы geonames можно скачать здесь http://download.geonames.org/export/dump/
- Тестовый датасет: https://disk.yandex.ru/d/wC296Rj3Yso2AQ

## 1. Загрузка и анализ данных 

### 1.1 Установка необходимых библиотек

In [1]:
pip install SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### 1.2 Загрузка необходимых библиотек

In [3]:
#общие
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')


#для работы с базами данных
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine, ARRAY, Float

#для преобразования в эмбединги
from sentence_transformers import SentenceTransformer

#для нахождения косинусной похожести
from sklearn.metrics.pairwise import cosine_similarity

#транслитерация из кириллицы в латиницу
from transliterate import translit

#поиск ближайших соседей
from scipy.spatial import cKDTree

### 1.3 Загрузка данных из базы данных

In [4]:
DATABASE = {
    'drivername': 'postgresql',
    'username': 'postgres', 
    'password': '******', 
    'host': 'localhost',
    'port': 5432,
    'database': 'geonames',
    'query': {}
}  

In [5]:
engine = create_engine(URL(**DATABASE))

#### Все необходимые для выполнения задания данные находятся в файле таблице geoname. Загрузим необходимые нам страны и отфильтруем города с населением более 15000.

In [6]:
query = "SELECT geonameid, name, alternatenames, country, admin1,\
       CASE \
WHEN country = 'RU' THEN 'RUSSIA' \
WHEN country = 'BY' THEN 'BELORUSSIA'\
WHEN country = 'AM' THEN 'ARMENIA'\
WHEN country = 'KZ' THEN 'KAZAHSTAN'\
WHEN country = 'KG' THEN 'KYRGYZSTAN'\
WHEN country = 'TR' THEN 'TURKEY' \
WHEN country = 'RS' THEN 'SERBIA' \
END AS country_ \
FROM geoname \
WHERE country IN ('RU', 'BY', 'AM', 'KZ','KG','TR','RS') and population > 15000" 

df = pd.read_sql_query(query, con=engine)

In [7]:
df

,geonameid,name,alternatenames,country,admin1,country_
0,483882,Tbilisskiy Rayon,"Tbilisskij Rajon,Tbilisskiy Rayon,Tbilissky Di...",RU,38,RUSSIA
1,483883,Tbilisskaya,"Tbilisskaja,Tbilisskaya,Tiflisskaya,Тбилисская",RU,38,RUSSIA
2,484004,Tatyshlinskiy Rayon,"Tatyshlinskij rajon,Tatyshlinsky District,Таты...",RU,08,RUSSIA
3,484016,Tatsinsky District,"Tacinskij rajon,Tatsinsky,Таци́нский райо́н",RU,61,RUSSIA
4,484048,Tatarstan,"Republique de Tatarstan,Respublika Tatarstan,R...",RU,73,RUSSIA
...,...,...,...,...,...,...
5214,483027,Tikhoretskiy Rayon,"Tikhoreckij Rajon,Tikhoreckij rajon,Tikhoretsk...",RU,38,RUSSIA
5215,483029,Tikhoretsk,"Cikharehck,Tichoreck,Tichoretsk,Tichorezk,Tich...",RU,38,RUSSIA
5216,483495,Terek,"Terek,Терек",RU,22,RUSSIA
5217,483661,Temryuk,"Temrjuk,Temryuk,Tyemryuk,Темрюк",RU,38,RUSSIA


### 1.4 Исследовательский анализ и обработка данных

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5219 entries, 0 to 5218
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   geonameid       5219 non-null   int64 
 1   name            5219 non-null   object
 2   alternatenames  4976 non-null   object
 3   country         5219 non-null   object
 4   admin1          5219 non-null   object
 5   country_        5219 non-null   object
dtypes: int64(1), object(5)
memory usage: 244.8+ KB


In [11]:
df[df.duplicated(['geonameid', 'name'])].count()

geonameid         1711
name              1711
alternatenames    1692
country           1711
admin1            1711
country_          1711
dtype: int64

Как видно  в столбце alternatenames присутствуют пропуски, а по основным столбцам - 'geonameid', 'name' есть дубликаты.
Регион определим из двух столбцов основной таблицы - 'country' и 'admin1'.

In [12]:
df['admin1'] = df['admin1'].astype(str)
df['country'] = df['country'].astype(str)

In [13]:
df['region'] = df['country']+ '.' + df['admin1'] 

Удалим ненужыне нам столбцы.

In [14]:
df = df.drop(['country', 'admin1'], axis=1)

In [15]:
df

,geonameid,name,alternatenames,country_,region
0,483882,Tbilisskiy Rayon,"Tbilisskij Rajon,Tbilisskiy Rayon,Tbilissky Di...",RUSSIA,RU.38
1,483883,Tbilisskaya,"Tbilisskaja,Tbilisskaya,Tiflisskaya,Тбилисская",RUSSIA,RU.38
2,484004,Tatyshlinskiy Rayon,"Tatyshlinskij rajon,Tatyshlinsky District,Таты...",RUSSIA,RU.08
3,484016,Tatsinsky District,"Tacinskij rajon,Tatsinsky,Таци́нский райо́н",RUSSIA,RU.61
4,484048,Tatarstan,"Republique de Tatarstan,Respublika Tatarstan,R...",RUSSIA,RU.73
...,...,...,...,...,...
5214,483027,Tikhoretskiy Rayon,"Tikhoreckij Rajon,Tikhoreckij rajon,Tikhoretsk...",RUSSIA,RU.38
5215,483029,Tikhoretsk,"Cikharehck,Tichoreck,Tichoretsk,Tichorezk,Tich...",RUSSIA,RU.38
5216,483495,Terek,"Terek,Терек",RUSSIA,RU.22
5217,483661,Temryuk,"Temrjuk,Temryuk,Tyemryuk,Темрюк",RUSSIA,RU.38


In [16]:
final_data = df.copy()

In [17]:
final_data.shape

(5219, 5)

In [18]:
final_data.isna().mean()

geonameid         0.000000
name              0.000000
alternatenames    0.046561
country_          0.000000
region            0.000000
dtype: float64

Для пропусков альтернативных названий заполним 'no information'.

In [19]:
final_data['alternatenames'] = final_data['alternatenames'].fillna('no information')

Избавимся от дубликатов по основным столбцам.

In [20]:
final_data = final_data.drop_duplicates(['geonameid','name'])

In [21]:
final_data.shape

(3508, 5)

## 2. Загрузка модели и получение эмбедингов названий городов

### 2.1 Для преобразования будем использовать предобученную модель LaBSE

In [22]:
model_new = SentenceTransformer('sentence-transformers/LaBSE')

Используем функцию удаляющую лишние символы и кодирующую названия городов

In [23]:
def preprocces_text(text: str) -> np.ndarray:
    '''
    Функция удаляет лишние символы и кодирует SentenceTransformer
    '''
    text = re.sub(r'[^\w\s\']', ' ', text)
    text = re.sub(r'[ \n]+', ' ', text).strip().lower()
    return model_new.encode(text)

### 2.2 Получение эмбедингов

In [56]:
final_data['embedding'] = final_data['name'].apply(preprocces_text)

In [153]:
final_data = final_data.reset_index(drop = True)

In [154]:
final_data

,geonameid,name,alternatenames,country_,region,embedding
0,483882,Tbilisskiy Rayon,"Tbilisskij Rajon,Tbilisskiy Rayon,Tbilissky Di...",RUSSIA,RU.38,"[0.048648756, 0.017007696, 0.056978054, -0.029..."
1,483883,Tbilisskaya,"Tbilisskaja,Tbilisskaya,Tiflisskaya,Тбилисская",RUSSIA,RU.38,"[0.0526159, 0.0085247755, 0.041342884, -0.0331..."
2,484004,Tatyshlinskiy Rayon,"Tatyshlinskij rajon,Tatyshlinsky District,Таты...",RUSSIA,RU.08,"[0.02076656, -0.02076974, 0.05847649, -0.04628..."
3,484016,Tatsinsky District,"Tacinskij rajon,Tatsinsky,Таци́нский райо́н",RUSSIA,RU.61,"[0.040978085, -0.0077150585, 0.049177878, -0.0..."
4,484048,Tatarstan,"Republique de Tatarstan,Respublika Tatarstan,R...",RUSSIA,RU.73,"[0.04620844, -0.02815654, 0.05455088, -0.04723..."
...,...,...,...,...,...,...
3503,480041,Tver Oblast,"Kalinin Oblast,Kalininskaya Oblast',Kalininska...",RUSSIA,RU.77,"[0.062859476, 0.024384918, 0.032452673, -0.056..."
3504,480508,Tula Oblast,"Oblast Tula,Oblast de Toula,Tul'skaja Oblast',...",RUSSIA,RU.76,"[0.044053428, 0.018218799, 0.00029260517, -0.0..."
3505,481956,Totsky District,"Tockij rajon,Totskiy Rayon,Totsky,То́цкий райо́н",RUSSIA,RU.55,"[0.020389335, -0.046109002, 0.033921458, -0.03..."
3506,482496,Tokarëvskiy Rayon,"Tokarevskij Rajon,Tokarevskiy Rayon,Tokaryovsk...",RUSSIA,RU.72,"[0.052405212, -0.05068418, 0.017585933, -0.055..."


In [155]:
final_data_str = final_data.copy()

### 2.3 Загрузка дата фрейма с эмбедингами в базу данных

In [156]:
final_data_str['embedding'] = final_data_str['embedding'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

In [157]:
#Загрузка дата-фрейма с эмбедингами в базу данных 
dtype = {'embedding': ARRAY(Float)}
final_data_str.to_sql(name='final_geonames', con=engine,  if_exists='replace', index=False, dtype=dtype)

508

### 2.4 Выгрузка дата фрейма с эмбедингами из базу данных

In [24]:
query_1 = "SELECT * \
           FROM final_geonames" 

final_data_str  =  pd.read_sql_query(query_1, con=engine)

In [25]:
final_data_str

,geonameid,name,alternatenames,country_,region,embedding
0,483882,Tbilisskiy Rayon,"Tbilisskij Rajon,Tbilisskiy Rayon,Tbilissky Di...",RUSSIA,RU.38,"[0.04864875599741936, 0.017007695510983467, 0...."
1,483883,Tbilisskaya,"Tbilisskaja,Tbilisskaya,Tiflisskaya,Тбилисская",RUSSIA,RU.38,"[0.052615899592638016, 0.008524775505065918, 0..."
2,484004,Tatyshlinskiy Rayon,"Tatyshlinskij rajon,Tatyshlinsky District,Таты...",RUSSIA,RU.08,"[0.02076655998826027, -0.020769739523530006, 0..."
3,484016,Tatsinsky District,"Tacinskij rajon,Tatsinsky,Таци́нский райо́н",RUSSIA,RU.61,"[0.0409780852496624, -0.007715058512985706, 0...."
4,484048,Tatarstan,"Republique de Tatarstan,Respublika Tatarstan,R...",RUSSIA,RU.73,"[0.04620844125747681, -0.028156539425253868, 0..."
...,...,...,...,...,...,...
3503,480041,Tver Oblast,"Kalinin Oblast,Kalininskaya Oblast',Kalininska...",RUSSIA,RU.77,"[0.06285947561264038, 0.024384917691349983, 0...."
3504,480508,Tula Oblast,"Oblast Tula,Oblast de Toula,Tul'skaja Oblast',...",RUSSIA,RU.76,"[0.04405342787504196, 0.01821879856288433, 0.0..."
3505,481956,Totsky District,"Tockij rajon,Totskiy Rayon,Totsky,То́цкий райо́н",RUSSIA,RU.55,"[0.020389335229992867, -0.04610900208353996, 0..."
3506,482496,Tokarëvskiy Rayon,"Tokarevskij Rajon,Tokarevskiy Rayon,Tokaryovsk...",RUSSIA,RU.72,"[0.052405212074518204, -0.05068418011069298, 0..."


### 2.5 Функция, осуществляющая мэтчинг вводимого названия города с имеющимися в базе данных

In [26]:
embeddings_new = final_data_str['embedding'].tolist()

In [27]:
# Строим дерево
city_tree = cKDTree(embeddings_new)

In [28]:
def get_most_similar(city_name: str):
    '''
    Функция принимает на вход название города и возвращает 5 различных городов, а также альтернативные названия  городов, 
    регион и страну, которые наиболее близки по косинусной похожести. Если город вводится на русском, 
    то сначала он преобразуется в латиницу.
    '''
    #Перевод в латиницу, если на кириллице
    translit_city_name = translit(city_name, 'ru', reversed=True)

    # Получение вектора для исправленного текста
    query_embedding = model_new.encode(translit_city_name)

    # Поиск ближайших соседей с использованием дерева
    distances, indices = city_tree.query(query_embedding, k=5)  # Задайте k равным количеству ближайших соседей

    # Получение ближайших городов
    similar_cities_df = final_data_str.iloc[indices]
    similar_cities = similar_cities_df[['geonameid', 'name', 'alternatenames', 'region', 'country_']]

    # Рассчитываем косинусную похожесть вместо расстояния
    similar_cities['cosine_similarity'] = 1 - distances / 2

    # Сортировка по убыванию косинусной похожести
    similar_cities = similar_cities.sort_values(by='cosine_similarity', ascending=False)

    result = {
        'input_city': city_name,
        'similar_cities': similar_cities.to_dict(orient='records'),
    }
    
    return result

## 3. Тестирование

### 3.1 Тестирование без ошибок в названии города

In [169]:
get_most_similar('Москва')

{'input_city': 'Москва',
 'similar_cities': [{'geonameid': 524901,
   'name': 'Moscow',
   'alternatenames': 'MOW,Maeskuy,Maskav,Maskava,Maskva,Mat-xco-va,Matxcova,Matxcơva,Mosca,Moscfa,Moscha,Mosco,Moscou,Moscova,Moscovo,Moscow,Moscoƿ,Moscu,Moscua,Moscòu,Moscó,Moscù,Moscú,Moskva,Moska,Moskau,Mosko,Moskokh,Moskou,Moskov,Moskova,Moskovu,Moskow,Moskowa,Mosku,Moskuas,Moskva,Moskve,Moskvo,Moskvy,Moskwa,Moszkva,Muskav,Musko,Mát-xcơ-va,Mòskwa,Məskeu,Məskəү,masko,maskw,mo si ke,moseukeuba,mosko,mosukuwa,mskw,mwskva,mwskw,mwsqbh,mx s ko,Μόσχα,Мæскуы,Маскав,Масква,Москва,Москве,Москвы,Москова,Москох,Москъва,Мускав,Муско,Мәскеу,Мәскәү,Մոսկվա,מאָסקװע,מאסקווע,מוסקבה,ماسکو,مسکو,موسكو,موسكۋا,ܡܘܣܩܒܐ,मास्को,मॉस्को,মস্কো,மாஸ்கோ,มอสโก,མོ་སི་ཁོ།,მოსკოვი,ሞስኮ,モスクワ,莫斯科,모스크바',
   'region': 'RU.48',
   'country_': 'RUSSIA',
   'cosine_similarity': 0.7579250200639593},
  {'geonameid': 857690,
   'name': 'Moskovskiy',
   'alternatenames': 'Moskovskij,Moskovskiy,Московский',
   'region': 'RU.48',
   'country_': 

In [29]:
get_most_similar('Ереван')

{'input_city': 'Ереван',
 'similar_cities': [{'geonameid': 616052,
   'name': 'Yerevan',
   'alternatenames': 'Ayrivan,Djerevan,EVN,Eireavan,Eireaván,Ereban,Erehvan,Ereun,Erevan,Erevan osh,Erevana,Erevano,Erevanum,Erevàn,Ereván,Erevāna,Erewan,Erivan,Eriwan,Erywan,Erywań,Gierevan,Ierevan,Iereván,Iravan,Jerevan,Jerevanas,Jerevani,Jereván,Jerewan,Jerjewan,Revan,Siro Mayraqaxaq Yerevan,Yerevan,Yervandavan,Yerêvan,Yiriwan,Yèrèvan,Yérévan,ayrwan,ereban,erevani,iyerebhana,shhr ayrwan,ye li wen,yeleban,yeravana pranta,yere wan,yerevan,yerevana,yerevhana,yryfan,yrywan,Èrevan,Êrîvan,İrəvan,Γιερεβάν,Ερεβάν,Єреван,Јереван,Ереван,Ереван ош,Ерэван,Երեվան,Երևան,יערעוואן,ירוואן,ایروان,شهر ایروان,يريفان,يېرېۋان,یریوان,یەریڤان,येरवान प्रान्त,येरेवान,येरेव्हान,ইয়েরেভান,யெரெவான்,เยเรวาน,ཡེ་རེ་ཝན།,ერევანი,ዬሬቫን,エレバン,葉里溫,예레반',
   'region': 'AM.11',
   'country_': 'ARMENIA',
   'cosine_similarity': 0.7156298764248072},
  {'geonameid': 616051,
   'name': 'Yerevan',
   'alternatenames': "Erevan,Ereván,Gorod Ye

In [31]:
get_most_similar('Antalya')

{'input_city': 'Antalya',
 'similar_cities': [{'geonameid': 323776,
   'name': 'Antalya',
   'alternatenames': "Antal'ja,Antali,Antalia,Antalija,Antalijos provincija,Antalijska provincie,Antalijská provincie,Antaljas ils,Antalya,Antalya Praant,Antalya Province,Antalya Vilayeti,Antalya Vilâyeti,Antalya eanangoddi,Antalya ili,Antalya probintzia,Antalya provints,Antalya walayati,Antalyan maakunta,Antalýa,Antália,Antʻaliayi nahang,Attalea,Eparchia Attaleias,Intara y'Antalya,Intara y’Antalya,Lalawigan ng Antalya,Mkoa wa Antalya,Pravincyja Antal'ja,Propinsi Antalya,Provinca Antalia,Provincia Antalya,Provincia dAntalya,Provincia de Antalya,Provincia di Adalia,Provinco Antalya,Provinsen Antalya,Provinsi Antalya,Província dAntalya,TR611,Vilojati Antalija,Wilayah Antalya,an ta li ya sheng,antallia ju,antalya,antalya pranta,antalya swbہ,antaruya xian,astan antalya,atali'a suba,yantalya vylayyty,Επαρχία Αττάλειας,Антали,Анталия,Анталија,Анталья,Вилояти Анталия,Правінцыя Анталья,Անթալիայի նահանգ,آن

In [33]:
get_most_similar('Bishkek')

{'input_city': 'Bishkek',
 'similar_cities': [{'geonameid': 1528675,
   'name': 'Bishkek',
   'alternatenames': 'Bichkek,Biscecum,Bischkek,Bishkek,Bishkek osh,Bisjkek,Biskek,Biskeka,Biskekas,Biskeko,Biskekʻ,Bisqeq,Bisqueque,Biszkek,Bixkek,Biŝkeko,Bişkek,Bişqeq,Biškek,Biškeka,Biškekas,Bișkek,Bîşkek,FRU,Frunze,Mpiskek,Pishkek,Pishpek,bi shen kai ke,bichkhek,bisakeka,bishukeku,biskek,biskeka,biskekk,bisyukekeu,bshkyk,byshkk,byshkyk,bysqq,picukkek,Μπισκέκ,Бишкек,Бишкек ош,Бішкек,Բիշկեք,בישקעק,בישקק,بشکیک,بيشكك,بيشكيك,بيشکک,بیشکک,بیشکێک,बिश्केक,বিশকেক,ਬਿਸ਼ਕੇਕ,பிசுக்கெக்,ಬಿಷ್ಕೆಕ್,ബിഷ്കെക്ക്,บิชเคก,པི་སི་ཁེག,ბიშკეკი,ቢሽኬክ,ビシュケク,比什凯克,비슈케크',
   'region': 'KG.01',
   'country_': 'KYRGYZSTAN',
   'cosine_similarity': 0.8623016737101795},
  {'geonameid': 1528334,
   'name': 'Gorod Bishkek',
   'alternatenames': 'Bishkek,Bishkek Shaary,Frunze,Gorod Bishkek,Gorod Frunze,Бишкек,Бишкек Шаары,Город Бишкек,Фрунзе',
   'region': 'KG.01',
   'country_': 'KYRGYZSTAN',
   'cosine_similarity': 0.6406469851563

In [35]:
get_most_similar('Актобе')

{'input_city': 'Актобе',
 'similar_cities': [{'geonameid': 610611,
   'name': 'Aktobe',
   'alternatenames': 'AKX,Aktioube,Aktioubé,Aktiube,Aktiubinsk,Aktjubinsk,Aktobe,Aktoebe,Aktubinsk,Aktyubinsk,Aktöbe,Aqtobe,Aqtoebe,Aqtóbe,Aqtöbe,Ukhtiubinskii,a ke tuo bi,agtoebe,akatobe,aktwby,aktʼobe,akutobe,aq\u200ctph  qzaqstan,ʼqtwbh,Актобе,Актюбинск,Ақтөбе,אקטובה,آق\u200cتپه، قزاقستان,أكتوبي,ਅਕਤੋਬੇ,აქტობე,アクトベ,阿克托比,악퇴베',
   'region': 'KZ.04',
   'country_': 'KAZAHSTAN',
   'cosine_similarity': 0.6927411855089645},
  {'geonameid': 324496,
   'name': 'Aksaray',
   'alternatenames': "Aksarai,Aksaraj,Aksarajus,Aksaray,Akusaraj,Aqserayiye,a ke sa lai,agsalai,akasara'e,aksarai,akusarai,aq sray,Ακσαράι,Аксарай,Аксарај,Акъсарай,Ақсарай,آق سراي,آق\u200cسرای,آکسرے,ਅਕਸਾਰਾਏ,აქსარაი,アクサライ,阿克萨赖,악사라이",
   'region': 'TR.75',
   'country_': 'TURKEY',
   'cosine_similarity': 0.5280162798349637},
  {'geonameid': 443185,
   'name': 'Aksaray',
   'alternatenames': "Aksaraj,Aksarajas ils,Aksaray,Aksaray Ili,Aksara

### 3.2 Тестирование с ошибками в названии города

In [36]:
get_most_similar('Масква')

{'input_city': 'Масква',
 'similar_cities': [{'geonameid': 524894,
   'name': 'Moskva',
   'alternatenames': 'Maskva,Moscou,Moscow,Moscu,Moscú,Moskau,Moskou,Moskovu,Moskva,Məskeu,Москва,Мәскеу',
   'region': 'RU.48',
   'country_': 'RUSSIA',
   'cosine_similarity': 0.6821996050705194},
  {'geonameid': 524901,
   'name': 'Moscow',
   'alternatenames': 'MOW,Maeskuy,Maskav,Maskava,Maskva,Mat-xco-va,Matxcova,Matxcơva,Mosca,Moscfa,Moscha,Mosco,Moscou,Moscova,Moscovo,Moscow,Moscoƿ,Moscu,Moscua,Moscòu,Moscó,Moscù,Moscú,Moskva,Moska,Moskau,Mosko,Moskokh,Moskou,Moskov,Moskova,Moskovu,Moskow,Moskowa,Mosku,Moskuas,Moskva,Moskve,Moskvo,Moskvy,Moskwa,Moszkva,Muskav,Musko,Mát-xcơ-va,Mòskwa,Məskeu,Məskəү,masko,maskw,mo si ke,moseukeuba,mosko,mosukuwa,mskw,mwskva,mwskw,mwsqbh,mx s ko,Μόσχα,Мæскуы,Маскав,Масква,Москва,Москве,Москвы,Москова,Москох,Москъва,Мускав,Муско,Мәскеу,Мәскәү,Մոսկվա,מאָסקװע,מאסקווע,מוסקבה,ماسکو,مسکو,موسكو,موسكۋا,ܡܘܣܩܒܐ,मास्को,मॉस्को,মস্কো,மாஸ்கோ,มอสโก,མོ་སི་ཁོ།,მოსკოვი,ሞስኮ,モスクワ,莫斯

In [37]:
get_most_similar('Иривен')

{'input_city': 'Иривен',
 'similar_cities': [{'geonameid': 616052,
   'name': 'Yerevan',
   'alternatenames': 'Ayrivan,Djerevan,EVN,Eireavan,Eireaván,Ereban,Erehvan,Ereun,Erevan,Erevan osh,Erevana,Erevano,Erevanum,Erevàn,Ereván,Erevāna,Erewan,Erivan,Eriwan,Erywan,Erywań,Gierevan,Ierevan,Iereván,Iravan,Jerevan,Jerevanas,Jerevani,Jereván,Jerewan,Jerjewan,Revan,Siro Mayraqaxaq Yerevan,Yerevan,Yervandavan,Yerêvan,Yiriwan,Yèrèvan,Yérévan,ayrwan,ereban,erevani,iyerebhana,shhr ayrwan,ye li wen,yeleban,yeravana pranta,yere wan,yerevan,yerevana,yerevhana,yryfan,yrywan,Èrevan,Êrîvan,İrəvan,Γιερεβάν,Ερεβάν,Єреван,Јереван,Ереван,Ереван ош,Ерэван,Երեվան,Երևան,יערעוואן,ירוואן,ایروان,شهر ایروان,يريفان,يېرېۋان,یریوان,یەریڤان,येरवान प्रान्त,येरेवान,येरेव्हान,ইয়েরেভান,யெரெவான்,เยเรวาน,ཡེ་རེ་ཝན།,ერევანი,ዬሬቫን,エレバン,葉里溫,예레반',
   'region': 'AM.11',
   'country_': 'ARMENIA',
   'cosine_similarity': 0.5864124238535362},
  {'geonameid': 616051,
   'name': 'Yerevan',
   'alternatenames': "Erevan,Ereván,Gorod Ye

In [39]:
get_most_similar('Stambull')

{'input_city': 'Stambull',
 'similar_cities': [{'geonameid': 487846,
   'name': 'Stavropol’',
   'alternatenames': "STW,Stavropol,Stavropol',Stavropol’,Stawropol,Voroshilovsk,Ставрополь",
   'region': 'RU.70',
   'country_': 'RUSSIA',
   'cosine_similarity': 0.5647358897653331},
  {'geonameid': 745042,
   'name': 'İstanbul',
   'alternatenames': 'Iostanbul,Iostanbúl,Ist,Istampoul,Istanbul,Istanbul Province,Istanbul Vilayeti,Miklagard,Miklagård,Provincia de Estambul,Stambul,TR100,astnbwl,isutanburu,yi si tan bu er,İst,İstanbul,İstanbul Vilâyeti,Ισταμπούλ,Истанбул,Стамбул,اسطنبول,イスタンブール,伊斯坦布尔',
   'region': 'TR.34',
   'country_': 'TURKEY',
   'cosine_similarity': 0.5178680036376425},
  {'geonameid': 745044,
   'name': 'Istanbul',
   'alternatenames': 'Bizanc,Bizánc,Byzance,Byzantion,Byzantium,Byzanz,Carigrad,Constantinoble,Constantinopla,Constantinople,Constantinopolen,Constantinopoli,Constantinopolis,Costantinopoli,Estambul,IST,Istamboul,Istambul,Istambuł,Istampoul,Istanbul,Istanbúl,I

In [51]:
get_most_similar('Остana')

{'input_city': 'Остana',
 'similar_cities': [{'geonameid': 539555,
   'name': 'Kulebaki',
   'alternatenames': 'Koelebaki,Koulebaki,Kulebaki,Kuljabaki,ku lie ba ji,kwlbaky,Кулебаки,Кулябакі,كولباكي,کولباکی,庫列巴基',
   'region': 'RU.51',
   'country_': 'RUSSIA',
   'cosine_similarity': 0.5947930183340395},
  {'geonameid': 514284,
   'name': 'Ostankinskiy',
   'alternatenames': 'Novoostankino,Ostankino,Ostankinskij,Ostankinskiy,Pushkinskoye,Останкинский',
   'region': 'RU.48',
   'country_': 'RUSSIA',
   'cosine_similarity': 0.5757596031211157},
  {'geonameid': 1538317,
   'name': 'Astana',
   'alternatenames': 'Astana,Astana Kˌalasy,Gorod Astana,Nur-Sultan,Nұr-Sұltan,nwr sltan,Астана,Астана Қаласы,Город Астана,Нур-Султан,Нұр-Сұлтан,نور سلطان',
   'region': 'KZ.05',
   'country_': 'KAZAHSTAN',
   'cosine_similarity': 0.5486915178625357},
  {'geonameid': 1526273,
   'name': 'Astana',
   'alternatenames': "Ak-Mola,Akmola,Akmolins'k,Akmolinsk,Aqmola,Astana,Astano,Astanà,Astaná,Asztana,Aστάνα,

In [50]:
get_most_similar('Сомарра')

{'input_city': 'Сомарра',
 'similar_cities': [{'geonameid': 300371,
   'name': 'Soma',
   'alternatenames': 'Distretto di Soma,Germe,Soma,Soma i Tyrkia,Somy,soma,suo ma,swmh,Σόμα,Сома,Сомы,סומה,ضلع سوما,ソマ,索馬,소마',
   'region': 'TR.45',
   'country_': 'TURKEY',
   'cosine_similarity': 0.6121149644939178},
  {'geonameid': 499099,
   'name': 'Samara',
   'alternatenames': 'KUF,Kuibyschew,Kuibyshev,Kujbyshev,Kuybyshev,Samar,Samar khot,Samar osh,Samara,Samarae,Samare,Samarga,Samaro,Samāra,Szamara,sa ma la,sa ma ra,samala,samara,smara,smara  rws,smrh,Σαμάρα,Куйбышев,Самар,Самар ош,Самар хот,Самарæ,Самара,Самаре,Самарҕа,Һамар,Սամարա,סמרה,سامارا,سامارہ,سمارا,سمارا، روس,समारा,ซามารา,სამარა,サマーラ,薩馬拉,사마라',
   'region': 'RU.65',
   'country_': 'RUSSIA',
   'cosine_similarity': 0.6118417980648657},
  {'geonameid': 3190342,
   'name': 'Sombor',
   'alternatenames': 'Coborszentmihaly,Coborszentmihály,Kiszombor,Mezozombor,Mezőzombor,Nagy-Zombor,Opstina Sombor,Opština Sombor,Schomburg,Sombar,Sombor,Som

In [61]:
get_most_similar('коленинград')

{'input_city': 'коленинград',
 'similar_cities': [{'geonameid': 554234,
   'name': 'Kaliningrad',
   'alternatenames': "Caliningrado,Calininopolis,KGD,Kalinin'nkrant,Kaliningrad,Kaliningrada,Kaliningradas,Kaliningrado,Kaliningradum,Kaliningrau,Kaliningráu,Kalininqrad,Kalinjingrad,Kalinyingrad,Kalinyingrád,Kalińingrad,Kalíníngrad,Karaliaucios,Karaliaucius,Karaliaučios,Karaliaučius,Kaļiņingrada,Kenisberg,Koenigsbarg,Koenigsberg,Koenigsberg in Preussen,Korigsberg,Krolewiec,Królewiec,Kënisberg,Königsbarg,Königsberg,Königsberg in Preußen,Körigsberg,jia li ning ge lei,kalininagrada,kalliningeuladeu,kalynynghrad,kalynyngrad,kariningurado,qlynyngrd,Καλίνινγκραντ,Калининград,Калињинград,Калінінград,Կալինինգրադ,קלינינגרד,كالينينغراد,کالیننگراڈ,کالینینگراد,کیلننگراڈ,कालिनिनग्राद,ಕಲಿನಿನ್\u200dಗ್ರಾಡ್,კალინინგრადი,カリーニングラード,加里寧格勒,칼리닌그라드",
   'region': 'RU.23',
   'country_': 'RUSSIA',
   'cosine_similarity': 0.6284742213674955},
  {'geonameid': 472757,
   'name': 'Volgograd',
   'alternatenames': "C

## 4. Вывод

Для решения поставленной задачи был использован метод преобразования названий городов в эмбединги и сравнение их с уже находящимися в базе данных. Для получения эмбедингов были протестированы 2 модели. Одна на основе предобученной модели 'distiluse-base-multilingual-cased-v2', вторая - 'LaBSE'. Лучший результат показала LaBSE. Как видно из приведенных тестов  разработанная функция хорошо справляется с мэчингов городов даже при наличиее нескольких ошибок в искомом названии города.